# Coding Variables


In [1]:
import pandas as pd

import numpy as np

In [2]:
db=pd.read_csv("database.csv", sep=";")

In [3]:
pd.set_option('display.max_columns', None)

# Création des variables

## Création de valeur

In [4]:
#Aide à la production de contenus
condition_aide_prod=[(db['B4[B41]']=='Oui, aide bénévole.')  | (db['B4[B42]']=='Oui, aide bénévole.') | (db['B4[B43]']=='Oui, aide bénévole.')| (db['B4[B44]']=='Oui, aide bénévole.') |(db['B4[B41]']=='Oui, aide rémunérée.')  | (db['B4[B42]']=='Oui, aide rémunérée.') | (db['B4[B43]']=='Oui, aide rémunérée.')| (db['B4[B44]']=='Oui, aide rémunérée.')]
values_aide_prod=['Yes']
db['help_prod']=np.select(condition_aide_prod, values_aide_prod)
db['help_prod']=db['help_prod'].replace('0', 'No')

#Aide à la gestion des contenus et communautés
conditions_aide_manag=[(db['B6[B61]']=='Oui, aide bénévole.')  | (db['B6[B62]']=='Oui, aide bénévole.') | (db['B6[B63]']=='Oui, aide bénévole.')|(db['B6[B61]']=='Oui, aide rémunérée.')  | (db['B6[B62]']=='Oui, aide rémunérée.') | (db['B6[B63]']=='Oui, aide rémunérée.')]
values_aide_manag=['Yes']
db['help_manag']=np.select(conditions_aide_manag, values_aide_manag)
db['help_manag']=db['help_manag'].replace('0', 'No')

#Membre du programme créateur de la plateforme
db = db.rename(columns={'B8[SQ001]': 'creator_prog'})
db['creator_prog']=db['creator_prog'].replace('Oui', 'Yes')
db['creator_prog']=db['creator_prog'].replace('Non', 'No')

#Fréquence de publication de contenu
condition_freq=[(db['B2']=='Plusieurs fois par semaine') |(db['B2']=='Tous les jours')]
value_freq=['Weekly']
db['freq']=np.select(condition_freq, value_freq)
db.loc[db['B2']=='Plusieurs fois par mois', 'freq']= 'Monthly'
db.loc[db['B2']=='Plus rarement', 'freq']= 'Rare'


#Format des contenus
condition_forma=[(db['B1[SQ001]']=='Oui') & (db['B1[SQ002]']=='Non'), 
                (db['B1[SQ001]']=='Non') & (db['B1[SQ002]']=='Oui'),
                (db['B1[SQ001]']=='Oui') & (db['B1[SQ002]']=='Oui')]
values_forma=['VOD_only', 'Live_only', 'VOD_Live']
db['format']=np.select(condition_forma, values_forma)

## Capture de valeur

In [5]:
#Part du revenu lié à la production sur la plateforme principale
db = db.rename(columns={'B15':'rev_share'})

#Revenu 2021
db = db.rename(columns={'E11':'rev_2021'})

#Les types de revenus
db = db.rename(columns={'B13[RB131]':'rev_ad', 
                       'B13[RB132]':'rev_subs', 
                       'B13[RB133]':'rev_sponso', 
                       'B13[RB134]':'rev_don_plat', 
                       'B13[RB135]':'rev_don_ext_plat', 
                       'B13[RB136]':'rev_prod_derives',
                       'B13[other]':'rev_other'})

db['rev_sponso'].fillna('No', inplace=True)

#Translation
db['rev_ad']=db['rev_ad'].replace('Oui', 'Yes')
db['rev_ad']=db['rev_ad'].replace('Non', 'No')

db['rev_subs']=db['rev_subs'].replace('Oui', 'Yes')
db['rev_subs']=db['rev_subs'].replace('Non', 'No')

db['rev_sponso']=db['rev_sponso'].replace('Oui', 'Yes')
db['rev_sponso']=db['rev_sponso'].replace('Non', 'No')

db['rev_share']=db['rev_share'].replace('Moins de 10%', 'Less than 10%')
db['rev_share']=db['rev_share'].replace('Entre 11% et 50%', 'Between 11% and 50%')
db['rev_share']=db['rev_share'].replace('Plus de 50%', 'More than 50%')

db['rev_2021']=db['rev_2021'].replace('Moins de 15 000€', 'Less than 15000€')
db['rev_2021']=db['rev_2021'].replace('Entre 15 000€ et 30 000€', 'Between 15000€ and 30000€')
db['rev_2021']=db['rev_2021'].replace('Plus de 30 000€', 'More than 30000€')

#Regroupement rémunération
db.loc[db['rev_don_plat']=='Oui', 'rev_don']= 'Yes'
db.loc[db['rev_don_ext_plat']=='Oui', 'rev_don']= 'Yes'



db.loc[db['rev_prod_derives']=='Oui', 'rev_other']= 'Yes'

In [6]:
db['rev_other'].fillna('No', inplace=True)

db.loc[db['rev_other']!='No', 'rev_other'] = 'Yes'

In [7]:
db['rev_don'].fillna('No', inplace=True)

## Value network

In [8]:
#Multihoming
db = db.rename(columns={'A2[A21]': 'prod_fb', 
                        'A2[A22]': 'prod_insta',
                       'A2[A23]': 'prod_tk', 
                        'A2[A24]': 'prod_tw', 
                       'A2[A25]': 'prod_yt', 
                        'A2[other]': 'prod_other'})

db['dummy_fb']=np.where(db['prod_fb']=='Oui',1,0)

db['dummy_insta']=np.where(db['prod_insta']=='Oui',1,0)

db['dummy_tk']=np.where(db['prod_tk']=='Oui',1,0)

db['dummy_tw']=np.where(db['prod_tw']=='Oui',1,0)

db['dummy_yt']=np.where(db['prod_yt']=='Oui',1,0)

db['nb_plat_prod']=db['dummy_fb']+db['dummy_insta']+db['dummy_tk']+db['dummy_tw']+db['dummy_yt']


condition_mh=[(db['nb_plat_prod']==1),
             (db['nb_plat_prod']>1)]
values_mh=['False', 'True']
db['MH']=np.select(condition_mh, values_mh)

## Duration 

In [9]:
db.loc[db['A6']=='Facebook', 'duration'] = db['A3[A21]']
db.loc[db['A6']=='Instagram', 'duration'] = db['A3[A22]']
db.loc[db['A6']=='TikTok', 'duration'] = db['A3[A23]']
db.loc[db['A6']=='Twitch', 'duration'] = db['A3[A24]']
db.loc[db['A6']=='Youtube', 'duration'] = db['A3[A25]']
db['duration'].value_counts()

Depuis cinq ans et plus.     61
Entre deux et quatres ans    42
Entre un et deux ans.        38
Depuis moins d'un an.        13
Name: duration, dtype: int64

In [10]:
db.loc[db['duration']=="Depuis moins d'un an.", 'duration']= 'Less than 2 years'
db.loc[db['duration']=='Entre un et deux ans.', 'duration']= 'Less than 2 years'

db.loc[db['duration']=='Entre deux et quatres ans', 'duration']= 'Between 2 and 4 years'
db.loc[db['duration']=='Depuis cinq ans et plus.', 'duration']= '5 years and more'

db['duration'].value_counts()

5 years and more         61
Less than 2 years        51
Between 2 and 4 years    42
Name: duration, dtype: int64

## Following

In [11]:
db.loc[db['A6']=='Facebook', 'follow'] = db['A4[A21]']
db.loc[db['A6']=='Instagram', 'follow'] = db['A4[A22]']
db.loc[db['A6']=='TikTok', 'follow'] = db['A4[A23]']
db.loc[db['A6']=='Twitch', 'follow'] = db['A4[A24]']
db.loc[db['A6']=='Youtube', 'follow'] = db['A4[A25]']


In [12]:
db.loc[db['follow']=='Moins de 1 000 personnes.', 'follow'] = 'Less than 10 000 followers'
db.loc[db['follow']=='Entre 1 000 et 10 000 personnes.', 'follow'] = 'Less than 10 000 followers'

db.loc[db['follow']=='Entre 10 000 et 100 000 personnes.', 'follow'] = 'More than 10 000 followers'
db.loc[db['follow']=='Plus de 100 000 personnes.', 'follow'] = 'More than 10 000 followers'
db['follow'].value_counts()

More than 10 000 followers    80
Less than 10 000 followers    74
Name: follow, dtype: int64

## Content

In [13]:
db.loc[db['A6']=='Facebook', 'topic'] = db['A5[A21]']
db.loc[db['A6']=='Instagram', 'topic'] = db['A5[A22]']
db.loc[db['A6']=='TikTok', 'topic'] = db['A5[A23]']
db.loc[db['A6']=='Twitch', 'topic'] = db['A5[A24]']
db.loc[db['A6']=='Youtube', 'topic'] = db['A5[A25]']

## Focus on only Twitch & YT

In [14]:
db_yt=db[(db['A6']=='Twitch') | (db['A6']=='Youtube')]

In [15]:
db_yt = db_yt.rename(columns={'A2[A21]': 'prod_fb', 
                        'A2[A22]': 'prod_insta',
                       'A2[A23]': 'prod_tk', 
                        'A2[A24]': 'prod_tw', 
                       'A2[A25]': 'prod_yt', 
                        'A2[other]': 'prod_other'})

#Platform distribution
df_plat_fb=db_yt.groupby(['prod_fb']).size().reset_index(name='counts')
df_plat_fb=df_plat_fb.rename(columns={'prod_fb':'platform'})
df_plat_fb=df_plat_fb.replace({'Oui': 'Facebook', 'Non':'N/A'})

df_plat_insta=db_yt.groupby(['prod_insta']).size().reset_index(name='counts')
df_plat_insta=df_plat_insta.rename(columns={'prod_insta':'platform'})
df_plat_insta=df_plat_insta.replace({'Oui': 'Instagram', 'Non':'N/A'})

df_plat_tk=db_yt.groupby(['prod_tk']).size().reset_index(name='counts')
df_plat_tk=df_plat_tk.rename(columns={'prod_tk':'platform'})
df_plat_tk=df_plat_tk.replace({'Oui': 'TikTok', 'Non':'N/A'})    


df_plat_tw=db_yt.groupby(['prod_tw']).size().reset_index(name='counts')
df_plat_tw=df_plat_tw.rename(columns={'prod_tw':'platform'})
df_plat_tw=df_plat_tw.replace({'Oui': 'Twitch', 'Non':'N/A'})

df_plat_yt=db_yt.groupby(['prod_yt']).size().reset_index(name='counts')
df_plat_yt=df_plat_yt.rename(columns={'prod_yt':'platform'})
df_plat_yt=df_plat_yt.replace({'Oui': 'Youtube', 'Non':'N/A'})


df_plat=pd.concat([df_plat_fb.query('platform == "Facebook"'), df_plat_insta.query('platform == "Instagram"'), df_plat_tk.query('platform == "TikTok"'), df_plat_tw.query('platform == "Twitch"'), df_plat_yt.query('platform == "Youtube"')], ignore_index=True)
df_plat

,platform,counts
0,Facebook,5
1,Instagram,43
2,TikTok,49
3,Twitch,90
4,Youtube,120


In [16]:
db_yt['duration'].value_counts()

5 years and more         59
Less than 2 years        48
Between 2 and 4 years    40
Name: duration, dtype: int64

In [17]:
db_yt['nb_plat_prod'].value_counts()

1    57
2    37
3    37
4    15
5     1
Name: nb_plat_prod, dtype: int64

In [18]:
db_yt['follow'].value_counts()

More than 10 000 followers    78
Less than 10 000 followers    69
Name: follow, dtype: int64

In [19]:
db_yt['topic'].value_counts()

Jeux Vidéo                               54
Éducatif / Vulgarisation Scientifique    36
Autre                                    22
Art et culture                           18
Comédie / Humour                          6
Sport                                     6
Lifestyle : mode, beauté                  2
Musique                                   2
Cuisine                                   1
Name: topic, dtype: int64

In [20]:
pd.crosstab(db_yt['format'], db_yt['freq'], margins=True)

freq,Monthly,Rare,Weekly,All
format,,,,
Live_only,5,3,48,56
VOD_Live,2,0,7,9
VOD_only,35,30,17,82
All,42,33,72,147


In [21]:
pd.crosstab(db_yt['A6'], db_yt['freq'], margins=True)

freq,Monthly,Rare,Weekly,All
A6,,,,
Twitch,5,3,50,58
Youtube,37,30,22,89
All,42,33,72,147


In [22]:
db_mca_other=db_yt[['help_prod', 'help_manag', 'freq', 'format', 'MH',
           'rev_ad', 'rev_subs', 'rev_sponso', 'rev_don', 'rev_other',
        'rev_share', 'rev_2021',
          'creator_prog', 'duration', 'follow']] 

db_mca_other.reset_index(drop=True)

,help_prod,help_manag,freq,format,MH,rev_ad,rev_subs,rev_sponso,rev_don,rev_other,rev_share,rev_2021,creator_prog,duration,follow
0,Yes,No,Weekly,Live_only,False,Yes,Yes,No,Yes,No,Less than 10%,Between 15000€ and 30000€,No,Less than 2 years,Less than 10 000 followers
1,No,No,Weekly,Live_only,False,No,Yes,No,Yes,No,Between 11% and 50%,Less than 15000€,Yes,Between 2 and 4 years,Less than 10 000 followers
2,No,No,Rare,Live_only,True,No,No,No,No,No,Less than 10%,Less than 15000€,No,Between 2 and 4 years,Less than 10 000 followers
3,Yes,Yes,Weekly,Live_only,True,Yes,Yes,No,Yes,No,Less than 10%,Between 15000€ and 30000€,Yes,Between 2 and 4 years,Less than 10 000 followers
4,Yes,Yes,Weekly,Live_only,False,No,Yes,No,Yes,No,Less than 10%,Less than 15000€,Yes,Less than 2 years,Less than 10 000 followers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Yes,Yes,Weekly,Live_only,True,Yes,Yes,No,Yes,No,Less than 10%,Less than 15000€,Yes,Less than 2 years,Less than 10 000 followers
143,No,No,Monthly,Live_only,False,No,Yes,No,No,No,Less than 10%,Less than 15000€,Yes,Less than 2 years,Less than 10 000 followers
144,Yes,Yes,Weekly,Live_only,True,Yes,Yes,Yes,Yes,Yes,More than 50%,Less than 15000€,Yes,5 years and more,More than 10 000 followers
145,Yes,Yes,Weekly,Live_only,True,Yes,Yes,Yes,No,No,Between 11% and 50%,More than 30000€,Yes,5 years and more,More than 10 000 followers


In [23]:
db_mca_other.to_csv('db_mca_other.csv', sep=';')

In [24]:
db_yt.to_csv('database_cleaned.csv', sep=';')

In [25]:
pd.crosstab(db_yt['E1'], db_yt['E2'], margins=True)

E2,De 18 à 25 ans,De 26 à 35 ans,De 36 à 50 ans,Plus de 50 ans,All
E1,,,,,
Non-binaire,0,3,0,0,3
Un homme,33,68,11,1,113
Une femme,6,20,5,0,31
All,39,91,16,1,147


In [26]:
db_mca=db_yt[['help_prod', 'help_manag', 'freq', 'format', 'MH',
           'rev_ad', 'rev_subs', 'rev_sponso', 'rev_don', 'rev_other',
        'rev_share', 'rev_2021',
          'creator_prog', 'duration', 'follow', 
                   'A6']] 

db_mca.reset_index(drop=True)
db_mca.to_csv('db_mca.csv', sep=";")